# YOLO Hand Keypoints Model Training (Google Colab)

This notebook trains a YOLO11n-pose model on the hand-keypoints dataset for gesture recognition.

**Dataset**: 26,768 images with 21 keypoints per hand
**Model**: yolo11n-pose.pt (nano - fastest)
**Output**: Trained model ready for use in Bin Diesel project


## Step 1: Setup and Install Dependencies


In [ ]:
# Install ultralytics (includes PyTorch)
!pip install ultralytics -q

# Verify GPU is available
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ WARNING: No GPU detected! Training will be very slow.")
    print("   Make sure Runtime > Change runtime type > Hardware accelerator = GPU")


## Step 2: Train the Model


In [ ]:
from ultralytics import YOLO
import os
import shutil

print("=" * 70)
print("YOLO Hand Keypoints Model Training")
print("=" * 70)
print("Dataset: hand-keypoints (26,768 images, 21 keypoints per hand)")
print("Model: yolo11n-pose.pt (nano - fastest)")
print()

# Load pretrained YOLO11n-pose model
model = YOLO("yolo11n-pose.pt")

# Determine device (use GPU if available)
device = 0 if torch.cuda.is_available() else 'cpu'
print(f"Using device: {'CUDA GPU' if device == 0 else 'CPU'}")
print()

# Training parameters
epochs = 100  # Full training on GPU
batch_size = 16  # Adjust if you run out of memory

print(f"Training configuration:")
print(f"  Epochs: {epochs}")
print(f"  Batch size: {batch_size}")
print(f"  Estimated time: ~2-4 hours on GPU")
print()
print("=" * 70)
print()

# Train the model
# The dataset will be automatically downloaded (369 MB)
results = model.train(
    data="hand-keypoints.yaml",  # Dataset YAML (auto-downloads if not found)
    epochs=epochs,
    imgsz=640,
    batch=batch_size,
    device=device,
    project="/content/runs/pose",
    name="hand_keypoints",
    exist_ok=True,
    patience=10,  # Early stopping if no improvement
    workers=8,
    cache=False,  # Don't cache images (saves memory)
    amp=True  # Mixed precision for faster training
)

print()
print("=" * 70)
print("Training Complete!")
print("=" * 70)


## Step 3: Save Model in Correct Format


In [ ]:
# Get the best model path from training results
best_model_path = os.path.join(results.save_dir, "weights", "best.pt")

# Create models directory structure (matching project structure)
models_dir = "/content/models/hand_keypoints/weights"
os.makedirs(models_dir, exist_ok=True)

# Copy best model to project structure
git_model_path = os.path.join(models_dir, "best.pt")
shutil.copy2(best_model_path, git_model_path)

print(f"Best model saved to: {best_model_path}")
print(f"Copied to project structure: {git_model_path}")
print()
print("Model file size:")
print(f"  {os.path.getsize(git_model_path) / 1e6:.1f} MB")
print()
print("✅ Model ready for download!")


## Step 4: Download the Trained Model


In [ ]:
from google.colab import files

# Download the trained model
print("Downloading trained model...")
files.download(git_model_path)

print("✅ Model downloaded!")
print()
print("Next steps:")
print("  1. Save the downloaded 'best.pt' file to your project:")
print("     models/hand_keypoints/weights/best.pt")
print("  2. Update config.py:")
print("     YOLO_HAND_MODEL = 'models/hand_keypoints/weights/best.pt'")
print("  3. Commit and push to git:")
print("     git add models/hand_keypoints/weights/best.pt")
print("     git commit -m 'Add trained hand-keypoints model'")
print("     git push origin main")


## Optional: Save to Google Drive

If you want to save the model to Google Drive instead of downloading:


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy model to Drive
drive_model_path = "/content/drive/MyDrive/bindiesel_hand_keypoints_best.pt"
shutil.copy2(git_model_path, drive_model_path)

print(f"✅ Model saved to Google Drive: {drive_model_path}")
